In [3]:
#Import necessary packages 
#To extract federal holidays ,we import the necessary classes 
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import datetime
import time
import pandas as pd

In [4]:
c  = pd.read_csv("Pi_seasons.csv")

In [5]:
c.head(2)

,DATE,AWND,PRCP,TMAX,TMIN,WT01,WT02,WT03,WT04,WT08,WT09,season
0,2015-04-01,4.70,0.00,58.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,spring
1,2015-04-02,11.41,0.18,69.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,spring


In [6]:
# create an instance of class custom business day  
us_calendar = CustomBusinessDay(calendar=USFederalHolidayCalendar()) 
# using the us_calendar as frequency to extract federal holidays
r = pd.date_range(start="1/1/2015",end="12/31/2018", freq=us_calendar)

In [7]:
#Create a function "holiday" and later map this function to  a new "holiday" column which returns "1" if it is a holiday or "0" if it is not.
def holiday(date):
    
    if date in r:
        return 0
    else:
        return 1
# Map the results to "DATE" column of  `dataframe`
c['holiday'] = c.DATE.map(holiday)

In [8]:
#Analyzing few rows of the dataframe
c.head()

,DATE,AWND,PRCP,TMAX,TMIN,WT01,WT02,WT03,WT04,WT08,WT09,season,holiday
0,2015-04-01,4.70,0.00,58.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,spring,0
1,2015-04-02,11.41,0.18,69.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,spring,0
2,2015-04-03,9.62,0.23,64.0,41.0,NaN,NaN,NaN,NaN,NaN,NaN,spring,0
3,2015-04-04,11.41,0.10,48.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,spring,1
4,2015-04-05,12.75,0.00,63.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,spring,1


In [9]:
c['DATE'] = pd.to_datetime(c['DATE'], format =  '%Y-%m-%d')

In [10]:
# Reading the "Rentals" csv
a = pd.read_csv("Pittsburgh_dataset_lon_lat.csv", header = 0)

In [11]:
a.head(1)

,trip_id,start_time,stop_time,bike_id,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,from_latitude,from_longitude,to_latitude,to_longitude
0,10676972,2015-05-31 06:54:00,2015-05-31 10:17:00,70166,12190,1018,Butler St & Stanton Ave,1001,Forbes Ave & Market Square,Customer,40.466103,-79.964628,40.440877,-80.00308


In [12]:
#To merge weather data to The Bikeshare rentals, We need just the datefield from the "Start_time" column , because it is in datetime format while the weather data’s date field is in date format. 
#To aid in merging dataframes, I created a temporary "DATE" column by stripping the time part of the date in "Renatals" df
a['DATE'] = pd.to_datetime(a['start_time']).dt.date

In [13]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269687 entries, 0 to 269686
Data columns (total 15 columns):
trip_id              269687 non-null int64
start_time           269687 non-null object
stop_time            269687 non-null object
bike_id              269687 non-null int64
trip_duration        269687 non-null int64
from_station_id      269687 non-null int64
from_station_name    269687 non-null object
to_station_id        269687 non-null int64
to_station_name      269687 non-null object
user_type            269687 non-null object
from_latitude        269687 non-null float64
from_longitude       269687 non-null float64
to_latitude          269687 non-null float64
to_longitude         269687 non-null float64
DATE                 269687 non-null object
dtypes: float64(4), int64(5), object(6)
memory usage: 30.9+ MB


In [14]:
# Coverting the "DATE" column to datetime format
a['DATE'] = pd.to_datetime(a['DATE'], format =  '%Y-%m-%d')

In [15]:
#Merging the two "Rentals" & "Seasons with holidays" dataframes together
df = pd.merge(a,c, on = ["DATE"])

In [18]:
df.head(5)

,trip_id,start_time,stop_time,bike_id,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,...,TMAX,TMIN,WT01,WT02,WT03,WT04,WT08,WT09,season,holiday
0,10676972,2015-05-31 06:54:00,2015-05-31 10:17:00,70166,12190,1018,Butler St & Stanton Ave,1001,Forbes Ave & Market Square,Customer,...,83.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN,spring,1
1,10684485,2015-05-31 09:39:00,2015-05-31 10:34:00,70289,3290,1018,Butler St & Stanton Ave,1001,Forbes Ave & Market Square,Customer,...,83.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN,spring,1
2,10686345,2015-05-31 10:20:00,2015-05-31 10:54:00,70368,2005,1018,Butler St & Stanton Ave,1001,Forbes Ave & Market Square,Customer,...,83.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN,spring,1
3,10686472,2015-05-31 10:23:00,2015-05-31 10:53:00,70365,1776,1018,Butler St & Stanton Ave,1001,Forbes Ave & Market Square,Customer,...,83.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN,spring,1
4,10687018,2015-05-31 10:35:00,2015-05-31 11:27:00,70167,3125,1018,Butler St & Stanton Ave,1001,Forbes Ave & Market Square,Customer,...,83.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN,spring,1


In [19]:
df.shape

(269687, 27)

In [21]:
df.to_csv("Pi_seasons_hol_latlon.csv",index = False)